In [18]:
!pip install datasets


In [19]:
from datasets import load_dataset

In [ ]:
persian_dataset = load_dataset("SeyedAli/Persian-Text-Sentiment")

In [ ]:
persian_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 55852
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 13964
    })
})

In [ ]:
min(persian_dataset['train']['label'])

0

In [ ]:
max(persian_dataset['train']['label'])

1

In [ ]:
for doc_index in range(0,10):
  print(f"[{doc_index}]\t label:{persian_dataset['train'][doc_index]['label']} \t text:{persian_dataset['train'][doc_index]['text'][0:100]}")

[0]	 label:1 	 text:این هندس فری تازه به دستم رسیده  و برای کسایی که انتظار شنیدن یک موسیقی با کیفیت دارن حتما پیشنهاد م
[1]	 label:1 	 text:عالی بود عالی، امیدوارم کیفیت همینجور بمونه
[2]	 label:1 	 text:من وقتی این ایرباد به لپ تاپ ایسوس  وصل میکنم و با نرم افزار حرفه ای   آهنگها را پخش میکنم   صدای گر
[3]	 label:1 	 text:با عرض سلام ضمن عرض خسته نباشید چلوکباب کوبیده سفارشی بسیار عالی بود ولی جوجه در چلو جوجه کباب مخصوص
[4]	 label:1 	 text:خوشمزه و داغ مخصوصا با سس چیلی که فرستاده بودن بسیار خوب بود ممنون ازجوجه داغ و اسنپ
[5]	 label:0 	 text:متاسفانه با وجود تاکید زیاد به خشک و برشته نبودن نان‌ها، نان سنگک به قدری خشک و بیسکویتی بود که قسمت
[6]	 label:0 	 text:طعم چیزکیک اوکی بود عالی نبود ولی اوکی بود اما بسته‌بندی و ارسالش بسیار نامناسب بود کیک کاملا از ریخ
[7]	 label:0 	 text:جوجه کباب خیلی پخته و خشک شده بود 
[8]	 label:0 	 text:سالاد فصل بسیار کیفیت بدی داشت وکاهوداخل ساندویچ از کیفیت خوبی برخوردارنبود
[9]	 label:0 	 text:من  تا مایع ظرفشویی سفارش دادم ولی یکی ارسال شد


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
vectorizer = TfidfVectorizer(strip_accents='unicode', min_df=50)
X = vectorizer.fit_transform(persian_dataset['train']['text'])

In [ ]:
vocab=vectorizer.get_feature_names_out()

In [ ]:
print(X.shape)

(55852, 1718)


In [ ]:
doc_vetors=X.toarray()

In [ ]:
# print(docvectors[0])

print([(i,vocab[i],j) for i,j in enumerate(doc_vetors[0]) if j>0])


[(24, 'ادم', 0.24028797492756176), (35, 'از', 0.0910560817005077), (104, 'انتظار', 0.20334508979874255), (118, 'انگار', 0.19780891959650423), (155, 'این', 0.34811923693755487), (172, 'با', 0.21158767676028226), (255, 'برای', 0.14323650481444672), (292, 'بعد', 0.1701550571058665), (313, 'به', 0.17819803083129337), (362, 'تا', 0.14644394192185362), (369, 'تازه', 0.14431559624961424), (480, 'حالا', 0.2263322534033856), (485, 'حتما', 0.21248400403999237), (587, 'دارن', 0.25763749971979744), (622, 'دستم', 0.17193635364194362), (705, 'رسیده', 0.26598658248933976), (995, 'فکر', 0.20429943497215733), (1236, 'میکنم', 0.1914453340466486), (1239, 'میکنه', 0.21957517271106466), (1518, 'پیشنهاد', 0.20164664093527157), (1596, 'کسایی', 0.30685367734222246), (1634, 'که', 0.0934175021725356), (1654, 'کیفیت', 0.10257485870167847), (1712, 'یک', 0.14249258040442211)]


# KMeans Clustering

In [21]:
from sklearn.cluster import KMeans
from time import time

In [ ]:
item_count=1000
km = KMeans(n_clusters=2, init='k-means++', max_iter=100)
t0 = time()
km.fit(X[:item_count])
labels=persian_dataset['train']['label'][:item_count]
print("done in %0.3fs" % (time() - t0))

done in 0.115s


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
from sklearn.metrics import  calinski_harabasz_score

In [ ]:
# Calculate clustering metric
#https://en.wikipedia.org/wiki/Calinski%E2%80%93Harabasz_index

ch_index = calinski_harabasz_score(X[:item_count].toarray(), km.labels_)

print(f"Calinski-Harabasz Index: {ch_index:.2f}")


Calinski-Harabasz Index: 10.98


In [ ]:
import pandas as pd
df = pd.DataFrame({"text":persian_dataset['train']['text'],"label":persian_dataset['train']['label']})

In [ ]:
df_cl=df[:item_count]

In [ ]:
df_cl['cluster'] = km.labels_

df_cl.head()



<ipython-input-20-2b3e031d655a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cl['cluster'] = km.labels_


,text,label,cluster
0,این هندس فری تازه به دستم رسیده و برای کسایی ...,1,0
1,عالی بود عالی، امیدوارم کیفیت همینجور بمونه,1,1
2,من وقتی این ایرباد به لپ تاپ ایسوس وصل میکنم ...,1,0
3,با عرض سلام ضمن عرض خسته نباشید چلوکباب کوبیده...,1,0
4,خوشمزه و داغ مخصوصا با سس چیلی که فرستاده بودن...,1,1


In [ ]:
# output the result to a text file.

clusters = df_cl.groupby('cluster')

In [ ]:
for cluster in clusters.groups:
    f = open('cluster'+str(cluster)+ '.csv', 'w') # create csv file
    data = clusters.get_group(cluster)[['text']] # get title and overview columns
    f.write(data.to_csv(index_label='id')) # set index to id
    f.close()


In [ ]:
print("Cluster centroids: \n")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

for i in range(2):
    print("Cluster %d:" % i)
    for j in order_centroids[i, :10]: #print out 10 feature terms of each cluster
        print (' %s' % terms[j])
    print('------------')

Cluster centroids: 

Cluster 0:
 به
 از
 بسیار
 سفارش
 بود
 با
 که
 غذا
 کیفیت
 من
------------
Cluster 1:
 بود
 خیلی
 عالی
 خوب
 هم
 خوشمزه
 زمینی
 سیب
 شده
 مرغ
------------


# persian_news_dataset

In [20]:
dataset = load_dataset("saied/persian_news_dataset", split="train",streaming=True)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for saied/persian_news_dataset contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/saied/persian_news_dataset
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [22]:
dataset

IterableDataset({
    features: ['text', 'title', 'category'],
    n_shards: 1
})

In [23]:
docs=[]
document_count=1000
counter=0
skip_counter=0
for doc in dataset:
    # print(f"{doc['category']}\t\t{doc['title']}\t\t\t\t{doc['text']}")
    if len(doc['title'])==0 or len(doc['category'])==0:
      skip_counter += 1
      if skip_counter%10000 ==0:
        print(f'{skip_counter} skipped')
      continue
    docs.append(doc)
    counter += 1
    if counter == document_count:
      break
print(len(docs))

10000 skipped
20000 skipped
30000 skipped
40000 skipped
50000 skipped
60000 skipped
70000 skipped
80000 skipped
90000 skipped
100000 skipped
110000 skipped
120000 skipped
130000 skipped
140000 skipped
150000 skipped
160000 skipped
170000 skipped
180000 skipped
190000 skipped
200000 skipped
210000 skipped
1000


In [24]:
docs[0]

{'text': 'الجزیره: نخست  وزیر رژیم صهیونیستی جنبش حماس را تهدید کرد\n........................................................تهران، خبرگزاری جمهوری اسلامی            06/03/86\nخارجی.سیاسی.غره.اسرائیل\n    بخش انگلیسی زبان شبکه تلویزیونی الجزیره قطر روز یکشنبه گزارش داد:\n"ایهوداولمرت" نخست وزیر رژیم صهیونیستی پس از اصابت موشک های  "قسام"\nمتعلق به گروه مسلح "حماس" اعلام کرد که هیچ یک از اعضای حماس در امان\nنخواهند بود.   الجزیره افزود: پس از آنکه رزمندگان فلسطینی روز یکشنبه دو فروند راکت\nاز غزه به سمت شهر "سدیروت" در جنوب اسراییل پرتاب کردند، اولمرت تهدید\nکرد که دامنه حملات نیروهای اسرائیلی را به غزه گسترش دهد.   وی همچنین در نشست کابینه رژیم صهیونیستی تاکید کرد که هیچ یک از\nاعضای وابسته به حماس درامان نخواهند بود.   به گزارش الجزیره: گردانهای عزالدین قسام شاخه نظامی حماس مسوولیت\nحمله موشکی امروز به جنوب فلسطین اشغالی را که به کشته شدن یک صهیونیست\nمنجر شد بر عهده گرفت.   براساس بیانیه گردانهای عزالدین قسام که برای رسانه ها ارسال شده است\nرزمندگان این گروه ساعت 15/9 دقیقه صبح امروز

In [25]:
len(set([doc['category'] for doc in docs]))

6

In [26]:
from sklearn.metrics import  calinski_harabasz_score

In [ ]:
!pip install stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 970.4/970.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 16.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-man

In [15]:
import stanza
stanza.download('fa')
nlp = stanza.Pipeline(lang='fa', processors='tokenize,pos')

preprocessed_docs = []
for doc in dataset:
    if len(doc['title']) == 0 or len(doc['category']) == 0:
        continue
    preprocessed_text = " ".join([word.text for sent in nlp(doc['text']).sentences for word in sent.words])
    preprocessed_docs.append({"text": preprocessed_text, "category": doc["category"]})


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: fa (Persian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/fa/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package      |
----------------------------
| tokenize  | perdt        |
| mwt       | perdt        |
| pos       | perdt_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


KeyboardInterrupt: 

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

min_df = 2
vectorizer = TfidfVectorizer(strip_accents='unicode', min_df=min_df)
X = vectorizer.fit_transform([doc["text"] for doc in preprocessed_docs])


In [64]:
from sklearn.cluster import KMeans
from time import time

item_count = 1000
km = KMeans(n_clusters=6, init='k-means++', max_iter=100)
t0 = time()
km.fit(X[:item_count])
print("done in %0.3fs" % (time() - t0))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


done in 0.811s


In [65]:
import pandas as pd

df = pd.DataFrame(preprocessed_docs[:item_count])
df['cluster'] = km.labels_

clusters = df.groupby('cluster')
for cluster in clusters.groups:
    f = open('cluster'+str(cluster)+ '.csv', 'w')
    data = clusters.get_group(cluster)[['text']]
    f.write(data.to_csv(index_label='id'))
    f.close()

In [66]:
from sklearn.metrics import calinski_harabasz_score

ch_index = calinski_harabasz_score(X[:item_count].toarray(), km.labels_)
print(f"Calinski-Harabasz Index: {ch_index:.2f}")

Calinski-Harabasz Index: 8.59


# Preprocess

preprocess Persian-Text-Sentiment


In [67]:
!pip install stanza

In [68]:
import stanza
stanza.download('fa')       # This downloads the English models for the neural pipeline

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: fa (Persian) ...
INFO:stanza:File exists: /root/stanza_resources/fa/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


In [76]:
nlp = stanza.Pipeline(lang='fa', processors='tokenize,pos')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package      |
----------------------------
| tokenize  | perdt        |
| mwt       | perdt        |
| pos       | perdt_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


In [77]:
doc = nlp("محسن محمدي متولد شهر تهران است. او در ۲۰ سالگی در مشهد به دانشگاه رفت.")

In [78]:
doc

[
  [
    {
      "id": 1,
      "text": "محسن",
      "upos": "PROPN",
      "xpos": "N_ANM",
      "feats": "Number=Sing",
      "start_char": 0,
      "end_char": 4
    },
    {
      "id": 2,
      "text": "محمدي",
      "upos": "PROPN",
      "xpos": "N_ANM",
      "feats": "Number=Sing",
      "start_char": 5,
      "end_char": 10
    },
    {
      "id": 3,
      "text": "متولد",
      "upos": "ADJ",
      "xpos": "ADJ_AJP",
      "start_char": 11,
      "end_char": 16
    },
    {
      "id": 4,
      "text": "شهر",
      "upos": "PROPN",
      "xpos": "N_IANM",
      "feats": "Number=Sing",
      "start_char": 17,
      "end_char": 20
    },
    {
      "id": 5,
      "text": "تهران",
      "upos": "PROPN",
      "xpos": "N_IANM",
      "feats": "Number=Sing",
      "start_char": 21,
      "end_char": 26
    },
    {
      "id": 6,
      "text": "است",
      "upos": "AUX",
      "xpos": "AUX",
      "feats": "Number=Sing|Person=3",
      "start_char": 27,
      "end_char": 30,

In [ ]:
print(*[f'word: {word.text+" "}\tpos: {word.pos
                                       ' for sent in doc.sentences for word in sent.words], sep='\n')

word: محسن 	pos: PROPN
word: محمدي 	pos: PROPN
word: متولد 	pos: ADJ
word: شهر 	pos: PROPN
word: تهران 	pos: PROPN
word: است 	pos: AUX
word: . 	pos: PUNCT
word: او 	pos: PRON
word: در 	pos: ADP
word: ۲۰ 	pos: NUM
word: سالگی 	pos: NOUN
word: در 	pos: ADP
word: مشهد 	pos: PROPN
word: به 	pos: ADP
word: دانشگاه 	pos: NOUN
word: رفت 	pos: VERB
word: . 	pos: PUNCT


Persian-Text-Sentiment

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score
import stanza

# Preprocessing function
def preprocess_text(text):
    nlp = stanza.Pipeline(lang='fa', processors='tokenize,pos')
    doc = nlp(text)
    filtered_tokens = [word.text for sent in doc.sentences for word in sent.words if word.pos in {'ADJ', 'ADV', 'NOUN', 'PRON', 'PROPN', 'VERB'}]
    return ' '.join(filtered_tokens)

# Load and preprocess the Persian-Text-Sentiment dataset
def load_and_preprocess_persian_text_sentiment(num_samples):
    from datasets import load_dataset
    dataset = load_dataset("SeyedAli/Persian-Text-Sentiment")

    preprocessed_texts = []
    counter = 0
    for doc in dataset['train']:
        preprocessed_text = preprocess_text(doc['text'])
        if preprocessed_text:
            preprocessed_texts.append(preprocessed_text)
            counter += 1
        if counter == num_samples:
            break
    return preprocessed_texts

# Function to perform clustering and evaluate
def perform_clustering(preprocessed_texts, min_df_val, num_clusters):
    vectorizer = TfidfVectorizer(strip_accents='unicode', min_df=min_df_val)
    X = vectorizer.fit_transform(preprocessed_texts)

    km = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=100)
    km.fit(X)

    ch_index = calinski_harabasz_score(X.toarray(), km.labels_)
    return km.labels_, ch_index



In [ ]:
# Parameters
min_df_values = [10]
item_count_values = [500]

best_ch_index = -1
best_params = {}

# Iterate over parameter combinations
for min_df_val in min_df_values:
    for item_count_val in item_count_values:
        preprocessed_texts = load_and_preprocess_persian_text_sentiment(item_count_val)
        labels, ch_index = perform_clustering(preprocessed_texts, min_df_val, 2)  # Two clusters for Persian-Text-Sentiment
        print(f"min_df: {min_df_val}, item_count: {item_count_val}, num_clusters: 2, CH Index: {ch_index}")
        if ch_index > best_ch_index:
            best_ch_index = ch_index
            best_params = {"min_df": min_df_val, "item_count": item_count_val, "num_clusters": 2}

print("Best Parameters:", best_params)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package      |
----------------------------
| tokenize  | perdt        |
| mwt       | perdt        |
| pos       | perdt_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package      |
----------------------------
| tokenize  | perdt        |
| mwt       | perdt        |
| pos       | perdt_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package      |
----------------------------
| tokenize  | perdt        |
| mwt       | perdt        |
| pos       | perdt_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package      |
----------------------------
| tokenize  | perdt        |
| mwt       | perdt        |
| pos       | perdt_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


...

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package      |
----------------------------
| tokenize  | perdt        |
| mwt       | perdt        |
| pos       | perdt_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package      |
----------------------------
| tokenize  | perdt        |
| mwt       | perdt        |
| pos       | perdt_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package      |
----------------------------
| tokenize  | perdt        |
| mwt       | perdt        |
| pos       | perdt_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package      |
----------------------------
| tokenize  | perdt        |
| mwt       | perdt        |
| pos       | perdt_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fa (Persian):
| Processor | Package      |
----------------------------
| tokenize  | perdt        |
| mwt       | perdt        |
| pos       | perdt_charlm |

